## DIA 027: Implementacion de Roles y Permisos en la API de Prediccion de Dígitos MNIST

**Resumen del Día 27: Implementación de Roles y Permisos en la API de Predicción de Dígitos MNIST**

En el **Día 27**, has avanzado en la **gestión y seguridad de usuarios** de tu API de predicción de dígitos MNIST implementando un sistema de **roles y permisos**. Esta funcionalidad permite diferenciar entre distintos tipos de usuarios (por ejemplo, administradores y usuarios regulares) y controlar el acceso a diferentes partes de la API según el rol asignado. Esto mejora la **seguridad**, **organización** y **usabilidad** de tu aplicación.

---

## **Índice**
1. [Objetivos del Día 27](#1-objetivos-del-día-27)
2. [Actualizar el Modelo de Usuario para Incluir Roles](#2-actualizar-el-modelo-de-usuario-para-incluir-roles)
3. [Crear Funciones de Decoradores para Roles](#3-crear-funciones-de-decoradores-para-roles)
4. [Implementar Endpoints Protegidos por Roles](#4-implementar-endpoints-protegidos-por-roles)
   - [4.1. Endpoint de Administración: Listar Usuarios](#41-endpoint-de-administración-listar-usuarios)
   - [4.2. Endpoint de Administración: Eliminar Usuario](#42-endpoint-de-administración-eliminar-usuario)
5. [Actualizar el Frontend para Manejar Roles](#5-actualizar-el-frontend-para-manejar-roles)
   - [5.1. Actualizar `index.html` para Funcionalidades de Administración](#51-actualizar-indexhtml-para-funcionalidades-de-administración)
6. [Actualizar las Pruebas Automatizadas para Roles y Permisos](#6-actualizar-las-pruebas-automatizadas-para-roles-y-permisos)
7. [Conclusiones y Recomendaciones](#7-conclusiones-y-recomendaciones)
8. [Recursos Adicionales](#8-recursos-adicionales)

---

## **1. Objetivos del Día 27**

1. **Definir Roles de Usuarios:**
   - Implementar diferentes roles (por ejemplo, `admin` y `user`) para categorizar a los usuarios.
   
2. **Controlar el Acceso Basado en Roles:**
   - Restringir el acceso a ciertos endpoints y funcionalidades según el rol del usuario.
   
3. **Actualizar el Modelo de Usuario:**
   - Modificar el modelo para incluir la información del rol.
   
4. **Crear Funciones de Decoradores para Roles:**
   - Facilitar la protección de endpoints mediante decoradores que verifiquen el rol del usuario.
   
5. **Actualizar el Frontend:**
   - Añadir interfaces y funcionalidades específicas para usuarios con roles administrativos.
   
6. **Actualizar las Pruebas Automatizadas:**
   - Asegurar que la implementación de roles y permisos funcione correctamente mediante pruebas.

---

## **2. Actualizar el Modelo de Usuario para Incluir Roles**

Para gestionar diferentes roles de usuarios, es necesario actualizar el modelo `User` para incluir un campo que almacene el rol asignado a cada usuario.

### **2.1. Modificar el Modelo `User`**

Actualiza el modelo `User` en tu archivo `api.py` para incluir el campo `role`.

```python
# api.py

from enum import Enum

class RoleEnum(Enum):
    ADMIN = 'admin'
    USER = 'user'

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    password = db.Column(db.String(128), nullable=False)
    email_confirmed = db.Column(db.Boolean, default=False)
    email_confirmed_on = db.Column(db.DateTime, nullable=True)
    role = db.Column(db.Enum(RoleEnum), default=RoleEnum.USER, nullable=False)  # Nuevo campo de rol

    def __repr__(self):
        return f'<User {self.username} - {self.role.value}>'
```

**Explicación:**

- **Enumeración `RoleEnum`:** Define los posibles roles que un usuario puede tener (`ADMIN` y `USER`).
- **Campo `role`:** Añade un campo `role` al modelo `User` que almacena el rol del usuario. Por defecto, se asigna el rol `USER`.

### **2.2. Actualizar la Base de Datos**

Después de modificar el modelo, es necesario crear y aplicar una nueva migración para actualizar la estructura de la base de datos.

```bash
flask db migrate -m "Añadir campo de rol al modelo de Usuario"
flask db upgrade
```

**Explicación:**

- **`flask db migrate`:** Genera una nueva migración basada en los cambios en los modelos.
- **`flask db upgrade`:** Aplica la migración a la base de datos, añadiendo el nuevo campo `role`.

---

## **3. Crear Funciones de Decoradores para Roles**

Para facilitar la protección de endpoints basados en roles, crearemos decoradores que verifiquen el rol del usuario antes de permitir el acceso.

### **3.1. Crear Decorador `role_required`**

Añade la siguiente función en tu archivo `api.py` para crear un decorador que verifique el rol del usuario.

```python
# api.py

from functools import wraps
from flask import request, jsonify
from flask_jwt_extended import get_jwt_identity

def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            
            if user.role.value != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            
            return f(*args, **kwargs)
        return wrapper
    return decorator
```

**Explicación:**

- **Función `role_required`:** Recibe el rol requerido para acceder al endpoint.
- **Decorador `decorator`:** Envoltorio que verifica si el usuario actual tiene el rol necesario.
- **Verificaciones:**
  - **Token de Acceso:** Asegura que el usuario haya proporcionado un token válido.
  - **Existencia del Usuario:** Verifica que el usuario exista en la base de datos.
  - **Rol del Usuario:** Comprueba que el rol del usuario coincide con el rol requerido.
  
- **Respuestas:**
  - **401 Unauthorized:** Si no se proporciona un token.
  - **404 Not Found:** Si el usuario no existe.
  - **403 Forbidden:** Si el usuario no tiene el rol adecuado.

---

## **4. Implementar Endpoints Protegidos por Roles**

Con los decoradores creados, podemos proteger ciertos endpoints para que solo sean accesibles por usuarios con roles específicos, como `admin`.

### **4.1. Endpoint de Administración: Listar Usuarios**

Crea un endpoint `/admin/users` que permita a los administradores listar todos los usuarios registrados.

```python
# api.py

@app.route('/admin/users', methods=['GET'])
@jwt_required()
@role_required('admin')
def list_users():
    """
    Endpoint para que los administradores listen todos los usuarios.
    """
    users = User.query.all()
    users_data = []
    for user in users:
        users_data.append({
            'id': user.id,
            'username': user.username,
            'email': user.email,
            'email_confirmed': user.email_confirmed,
            'role': user.role.value
        })
    return jsonify(users_data), 200
```

**Explicación:**

- **Decoradores:**
  - **`@jwt_required()`:** Asegura que el usuario esté autenticado.
  - **`@role_required('admin')`:** Restringe el acceso a usuarios con rol `admin`.
  
- **Funcionalidad:**
  - Recupera todos los usuarios de la base de datos.
  - Formatea los datos de los usuarios para la respuesta.
  
- **Respuesta:**
  - **200 OK:** Lista de usuarios con sus detalles.

### **4.2. Endpoint de Administración: Eliminar Usuario**

Crea un endpoint `/admin/users/<int:user_id>` que permita a los administradores eliminar un usuario específico.

```python
# api.py

@app.route('/admin/users/<int:user_id>', methods=['DELETE'])
@jwt_required()
@role_required('admin')
def delete_user(user_id):
    """
    Endpoint para que los administradores eliminen un usuario específico.
    """
    user = User.query.get_or_404(user_id)
    db.session.delete(user)
    db.session.commit()
    return jsonify({"msg": f"Usuario '{user.username}' eliminado exitosamente."}), 200
```

**Explicación:**

- **Decoradores:**
  - **`@jwt_required()`:** Asegura que el usuario esté autenticado.
  - **`@role_required('admin')`:** Restringe el acceso a usuarios con rol `admin`.
  
- **Funcionalidad:**
  - Busca al usuario por `user_id`. Si no existe, retorna un error 404.
  - Elimina al usuario de la base de datos.
  
- **Respuesta:**
  - **200 OK:** Mensaje confirmando la eliminación del usuario.

---

## **5. Actualizar el Frontend para Manejar Roles**

Para permitir que los administradores gestionen usuarios desde el frontend, es necesario actualizar la interfaz de usuario.

### **5.1. Actualizar `index.html` para Funcionalidades de Administración**

Añade secciones y formularios en tu archivo `templates/index.html` que permitan a los administradores listar y eliminar usuarios.

```html
<!-- templates/index.html -->

<!DOCTYPE html>
<html>
<head>
    <title>Clasificador de Dígitos MNIST</title>
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>

    <h1>Clasificador de Dígitos MNIST</h1>
    
    <!-- Formulario de Registro -->
    <h2>Registrar Usuario</h2>
    <form id="registerForm">
        <label for="reg_username">Usuario:</label>
        <input type="text" id="reg_username" name="username" required>
        <br>
        <label for="reg_email">Email:</label>
        <input type="email" id="reg_email" name="email" required>
        <br>
        <label for="reg_password">Contraseña:</label>
        <input type="password" id="reg_password" name="password" required>
        <br>
        <button type="submit">Registrar</button>
    </form>
    
    <div id="registerResult"></div>
    
    <!-- Formulario de Login -->
    <h2>Iniciar Sesión</h2>
    <form id="loginForm">
        <label for="username">Usuario:</label>
        <input type="text" id="username" name="username" required>
        <br>
        <label for="password">Contraseña:</label>
        <input type="password" id="password" name="password" required>
        <br>
        <button type="submit">Iniciar Sesión</button>
    </form>
    
    <div id="loginResult"></div>
    
    <!-- Formulario de Solicitud de Restablecimiento de Contraseña -->
    <h2>¿Olvidaste tu Contraseña?</h2>
    <form id="resetRequestForm">
        <label for="reset_email">Ingresa tu Email:</label>
        <input type="email" id="reset_email" name="email" required>
        <br>
        <button type="submit">Restablecer Contraseña</button>
    </form>
    
    <div id="resetRequestResult"></div>
    
    <!-- Formulario de Restablecimiento de Contraseña -->
    <h2>Restablecer Contraseña</h2>
    <form id="resetPasswordForm">
        <label for="new_password">Nueva Contraseña:</label>
        <input type="password" id="new_password" name="password" required>
        <br>
        <button type="submit">Actualizar Contraseña</button>
    </form>
    
    <div id="resetPasswordResult"></div>
    
    <!-- Sección de Administración -->
    <h2>Administración de Usuarios</h2>
    <button onclick="listUsers()">Listar Usuarios</button>
    <div id="adminUsersList"></div>
    
    <script>
        let accessToken = '';

        // Manejar el formulario de registro
        $('#registerForm').submit(function(event) {
            event.preventDefault();
            const username = $('#reg_username').val();
            const email = $('#reg_email').val();
            const password = $('#reg_password').val();

            $.ajax({
                url: '/register',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, email: email, password: password }),
                success: function(response) {
                    $('#registerResult').text('Registro exitoso. Por favor, confirma tu email.');
                },
                error: function(xhr, status, error) {
                    $('#registerResult').text('Error en el registro: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de login
        $('#loginForm').submit(function(event) {
            event.preventDefault();
            const username = $('#username').val();
            const password = $('#password').val();

            $.ajax({
                url: '/login',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ username: username, password: password }),
                success: function(response) {
                    accessToken = response.access_token;
                    $('#loginResult').text('Inicio de sesión exitoso. Token obtenido.');
                },
                error: function(xhr, status, error) {
                    $('#loginResult').text('Error en el inicio de sesión: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de solicitud de restablecimiento de contraseña
        $('#resetRequestForm').submit(function(event) {
            event.preventDefault();
            const email = $('#reset_email').val();

            $.ajax({
                url: '/reset_password_request',
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ email: email }),
                success: function(response) {
                    $('#resetRequestResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    $('#resetRequestResult').text('Error: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Manejar el formulario de restablecimiento de contraseña
        $('#resetPasswordForm').submit(function(event) {
            event.preventDefault();
            const urlParams = new URLSearchParams(window.location.search);
            const token = window.location.pathname.split('/').pop();
            const new_password = $('#new_password').val();

            $.ajax({
                url: `/reset_password/${token}`,
                type: 'POST',
                contentType: 'application/json',
                data: JSON.stringify({ password: new_password }),
                success: function(response) {
                    $('#resetPasswordResult').text(response.msg);
                },
                error: function(xhr, status, error) {
                    $('#resetPasswordResult').text('Error: ' + xhr.responseJSON.msg);
                }
            });
        });

        // Función para listar usuarios (administradores)
        function listUsers() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            $.ajax({
                url: '/admin/users',
                type: 'GET',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                success: function(response) {
                    let usersHtml = '<ul>';
                    response.forEach(user => {
                        usersHtml += `<li>ID: ${user.id} | Usuario: ${user.username} | Email: ${user.email} | Confirmado: ${user.email_confirmed} | Rol: ${user.role}
                                      <button onclick="deleteUser(${user.id})">Eliminar</button></li>`;
                    });
                    usersHtml += '</ul>';
                    $('#adminUsersList').html(usersHtml);
                },
                error: function(xhr, status, error) {
                    $('#adminUsersList').text('Error: ' + xhr.responseJSON.msg);
                }
            });
        }

        // Función para eliminar un usuario (administradores)
        function deleteUser(userId) {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            if (!confirm("¿Estás seguro de que deseas eliminar este usuario?")) {
                return;
            }

            $.ajax({
                url: `/admin/users/${userId}`,
                type: 'DELETE',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                success: function(response) {
                    alert(response.msg);
                    listUsers();  // Actualizar la lista de usuarios
                },
                error: function(xhr, status, error) {
                    alert('Error: ' + xhr.responseJSON.msg);
                }
            });
        }

        function uploadImage() {
            if (!accessToken) {
                alert("Por favor, inicia sesión primero.");
                return;
            }

            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            if (!file) {
                alert("Por favor, selecciona una imagen.");
                return;
            }

            var formData = new FormData();
            formData.append('file', file);

            $.ajax({
                url: '/predict',
                type: 'POST',
                headers: {
                    'Authorization': 'Bearer ' + accessToken
                },
                data: formData,
                contentType: false,
                processData: false,
                success: function(response) {
                    if(response.error){
                        $('#result').text('Error: ' + response.error);
                    } else {
                        let cacheStatus = response.cached ? ' (Desde Cache)' : '';
                        $('#result').text('Predicción: ' + response.prediccion + ' | Probabilidad: ' + (response.probabilidad * 100).toFixed(2) + '%' + cacheStatus);
                    }
                },
                error: function(xhr, status, error) {
                    $('#result').text('Error: ' + xhr.responseJSON.error);
                }
            });
        }
    </script>

</body>
</html>
```

**Explicación:**

- **Sección de Administración:**
  - **Botón "Listar Usuarios":** Permite a los administradores recuperar y visualizar todos los usuarios registrados.
  - **Botones "Eliminar":** Junto a cada usuario en la lista, se añade un botón para eliminar al usuario correspondiente.
  
- **Funciones de JavaScript:**
  - **`listUsers()`:** Envía una solicitud GET al endpoint `/admin/users` para obtener la lista de usuarios y la muestra en la página.
  - **`deleteUser(userId)`:** Envía una solicitud DELETE al endpoint `/admin/users/<user_id>` para eliminar un usuario específico.

**Nota:** Para una mejor experiencia, considera separar las funcionalidades administrativas en una página dedicada y proteger adecuadamente el acceso en el frontend.

---

## **6. Actualizar las Pruebas Automatizadas para Roles y Permisos**

Es crucial asegurar que la implementación de roles y permisos funcione correctamente mediante pruebas automatizadas.

### **6.1. Actualizar `test_api.py` para Incluir Pruebas de Roles**

Añade las siguientes pruebas en `tests/test_api.py` para verificar que solo los administradores puedan acceder a ciertos endpoints.

```python
# tests/test_api.py

import pytest
from api import app, db, User, RoleEnum, generate_confirmation_token, confirm_token
import os
from io import BytesIO
from PIL import Image
import numpy as np
import json
from flask import url_for
from unittest.mock import patch

@pytest.fixture
def client():
    app.config['TESTING'] = True
    app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///:memory:'  # Usar una base de datos en memoria para pruebas
    app.config['MAIL_SUPPRESS_SEND'] = True  # Evita el envío de emails durante las pruebas
    with app.test_client() as client:
        with app.app_context():
            db.create_all()
            # Crear un usuario admin para pruebas
            admin_user = User(
                username='admin',
                email='admin@example.com',
                password=bcrypt.generate_password_hash('adminpassword').decode('utf-8'),
                email_confirmed=True,
                role=RoleEnum.ADMIN
            )
            db.session.add(admin_user)
            db.session.commit()
            yield client
            db.session.remove()
            db.drop_all()

def generate_image(digit=5):
    # Genera una imagen en blanco con un dígito negro en el centro
    img = Image.new('L', (28, 28), color=255)
    return img

def register_user(client, username, email, password, role=RoleEnum.USER):
    response = client.post('/register', json={
        'username': username,
        'email': email,
        'password': password
    })
    # Actualizar el rol si es necesario
    if role != RoleEnum.USER:
        user = User.query.filter_by(username=username).first()
        user.role = role
        user.email_confirmed = True  # Confirmar email para simplificar
        db.session.commit()
    return response

def login(client, username, password):
    response = client.post('/login', json={
        'username': username,
        'password': password
    })
    data = response.get_json()
    return data.get('access_token')

@patch('api.mail.send')
def test_admin_access(mock_send, client):
    # Registrar un usuario regular
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword', role=RoleEnum.USER)
    
    # Registrar un usuario admin
    register_user(client, 'adminuser', 'adminuser@example.com', 'adminpassword', role=RoleEnum.ADMIN)
    
    # Iniciar sesión como usuario regular
    user_token = login(client, 'testuser', 'testpassword')
    assert user_token is not None
    
    # Iniciar sesión como admin
    admin_token = login(client, 'adminuser', 'adminpassword')
    assert admin_token is not None
    
    # Usuario regular intenta listar usuarios (debe fallar)
    response = client.get('/admin/users', headers={'Authorization': f'Bearer {user_token}'})
    assert response.status_code == 403
    json_data = response.get_json()
    assert json_data['msg'] == 'Acceso no autorizado'
    
    # Admin lista usuarios (debe tener éxito)
    response = client.get('/admin/users', headers={'Authorization': f'Bearer {admin_token}'})
    assert response.status_code == 200
    json_data = response.get_json()
    assert isinstance(json_data, list)
    assert len(json_data) >= 2  # Al menos el admin y testuser
    
@patch('api.mail.send')
def test_admin_delete_user(mock_send, client):
    # Registrar un usuario regular
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword', role=RoleEnum.USER)
    
    # Iniciar sesión como admin
    admin_token = login(client, 'admin', 'adminpassword')
    assert admin_token is not None
    
    # Obtener el ID del usuario a eliminar
    user = User.query.filter_by(username='testuser').first()
    assert user is not None
    user_id = user.id
    
    # Admin elimina al usuario
    response = client.delete(f'/admin/users/{user_id}', headers={'Authorization': f'Bearer {admin_token}'})
    assert response.status_code == 200
    json_data = response.get_json()
    assert json_data['msg'] == f"Usuario 'testuser' eliminado exitosamente."
    
    # Verificar que el usuario ya no existe
    user = User.query.filter_by(username='testuser').first()
    assert user is None

def test_non_admin_cannot_access_admin_endpoints(client):
    # Registrar un usuario regular
    register_user(client, 'testuser', 'testuser@example.com', 'testpassword', role=RoleEnum.USER)
    
    # Iniciar sesión como usuario regular
    user_token = login(client, 'testuser', 'testpassword')
    assert user_token is not None
    
    # Usuario regular intenta eliminar un usuario (debe fallar)
    response = client.delete('/admin/users/1', headers={'Authorization': f'Bearer {user_token}'})
    assert response.status_code == 403
    json_data = response.get_json()
    assert json_data['msg'] == 'Acceso no autorizado'
```

**Explicación:**

- **Fixture `client`:** Configura una base de datos en memoria y crea un usuario administrador para las pruebas.
  
- **Funciones Auxiliares:**
  - **`register_user`:** Registra un usuario con un rol específico. Si el rol es diferente de `USER`, asigna el rol correspondiente y marca el email como confirmado.
  - **`login`:** Inicia sesión y devuelve el token JWT.
  
- **Pruebas Añadidas:**
  - **`test_admin_access`:**
    - Registra un usuario regular y un administrador.
    - Verifica que un usuario regular no pueda acceder a endpoints administrativos.
    - Verifica que un administrador pueda listar usuarios.
    
  - **`test_admin_delete_user`:**
    - Registra y elimina a un usuario regular utilizando un token de administrador.
    - Verifica que el usuario haya sido eliminado correctamente.
    
  - **`test_non_admin_cannot_access_admin_endpoints`:**
    - Verifica que un usuario regular no pueda eliminar usuarios, protegiendo así los endpoints administrativos.

---

## **7. Conclusiones y Recomendaciones**

### **7.1. Implementación de Roles y Permisos**

- **Beneficios:**
  - **Seguridad Mejorada:** Restringe el acceso a funcionalidades sensibles solo a usuarios autorizados.
  - **Organización y Escalabilidad:** Facilita la gestión de usuarios con diferentes niveles de acceso a medida que la aplicación crece.
  
- **Recomendaciones:**
  - **Ampliar Roles:** Considera la posibilidad de añadir más roles según las necesidades de la aplicación (por ejemplo, `moderator`, `editor`).
  - **Gestión Dinámica de Roles:** Implementa funcionalidades que permitan asignar y cambiar roles desde el frontend o mediante endpoints administrativos.
  
### **7.2. Mejorar la Experiencia del Usuario en el Frontend**

- **Interfaz de Administración:**
  - Crear una interfaz más intuitiva y atractiva para la administración de usuarios, posiblemente separando esta sección en una página dedicada.
  
- **Mensajes Claros y Feedback:**
  - Asegurar que los usuarios reciban mensajes claros y útiles durante las operaciones de registro, login, restablecimiento de contraseña y administración.
  
### **7.3. Optimizar la Seguridad Adicional**

- **Protección contra Ataques Comunes:**
  - Implementar medidas contra inyecciones SQL, Cross-Site Scripting (XSS) y Cross-Site Request Forgery (CSRF).
  
- **Monitoreo y Registro:**
  - Añadir funcionalidades de monitoreo y registro para detectar y responder a actividades sospechosas.
  
### **7.4. Ampliar las Pruebas Automatizadas**

- **Cobertura Completa:**
  - Continuar ampliando las pruebas para cubrir todos los casos de uso, incluyendo la asignación y revocación de roles.
  
- **Pruebas de Integración y Seguridad:**
  - Implementar pruebas que evalúen la interacción entre diferentes componentes de la aplicación y la resistencia ante intentos de vulneración.

---

## **8. Recursos Adicionales**

- **Flask Documentation:** [Flask Docs](https://flask.palletsprojects.com/en/2.2.x/)
- **Flask-JWT-Extended Documentation:** [Flask-JWT-Extended Docs](https://flask-jwt-extended.readthedocs.io/en/stable/)
- **Flask-Mail Documentation:** [Flask-Mail Docs](https://pythonhosted.org/Flask-Mail/)
- **SQLAlchemy Documentation:** [SQLAlchemy Docs](https://docs.sqlalchemy.org/en/20/)
- **Flask-SQLAlchemy Documentation:** [Flask-SQLAlchemy Docs](https://flask-sqlalchemy.palletsprojects.com/en/3.0.x/)
- **Flask-Migrate Documentation:** [Flask-Migrate Docs](https://flask-migrate.readthedocs.io/en/latest/)
- **Flask-Bcrypt Documentation:** [Flask-Bcrypt Docs](https://flask-bcrypt.readthedocs.io/en/latest/)
- **PyTest Documentation:** [PyTest Docs](https://docs.pytest.org/en/7.1.x/)
- **itsdangerous Documentation:** [itsdangerous Docs](https://pythonhosted.org/itsdangerous/)
- **Flask-Limiter Documentation:** [Flask-Limiter Docs](https://flask-limiter.readthedocs.io/en/stable/)
- **OWASP Authorization Cheat Sheet:** [OWASP Authorization Cheat Sheet](https://cheatsheetseries.owasp.org/cheatsheets/Authorization_Cheat_Sheet.html)
- **GitHub Actions Documentation:** [GitHub Actions Docs](https://docs.github.com/en/actions)

---

## **Conclusión**

En el **Día 27**, has implementado un sistema de **roles y permisos** que permite diferenciar entre distintos tipos de usuarios (por ejemplo, administradores y usuarios regulares) y controlar el acceso a ciertas funcionalidades de la API según el rol asignado. Esta implementación mejora la **seguridad** y **organización** de tu aplicación, permitiendo gestionar usuarios de manera más eficiente y proteger endpoints sensibles contra accesos no autorizados.

**Pasos Clave Realizados:**

1. **Actualización del Modelo de Usuario:**
   - Añadiste un campo `role` al modelo `User` para almacenar el rol de cada usuario.
   
2. **Creación de Decoradores para Roles:**
   - Implementaste el decorador `role_required` para facilitar la protección de endpoints basados en roles.
   
3. **Implementación de Endpoints Administrativos:**
   - Creaste endpoints que solo pueden ser accedidos por usuarios con rol `admin`, como listar y eliminar usuarios.
   
4. **Actualización del Frontend:**
   - Añadiste funcionalidades en el frontend para que los administradores puedan gestionar usuarios desde la interfaz.
   
5. **Actualización de Pruebas Automatizadas:**
   - Incluiste pruebas que verifican que los roles y permisos funcionan correctamente, asegurando que solo los administradores puedan acceder a ciertas funcionalidades.

**Recomendaciones para Continuar:**

- **Implementar Roles Adicionales:**
  - Considera añadir más roles según las necesidades de la aplicación para una gestión más granular.
  
- **Mejorar la Gestión de Roles:**
  - Implementa endpoints o funcionalidades que permitan asignar y cambiar roles desde el frontend o mediante una interfaz de administración.
  
- **Optimizar la Seguridad:**
  - Añade medidas adicionales de seguridad como la protección contra ataques CSRF y la validación exhaustiva de entradas.
  
- **Ampliar las Pruebas Automatizadas:**
  - Continúa desarrollando pruebas para cubrir todas las funcionalidades relacionadas con roles y permisos, así como otros aspectos de la seguridad de la aplicación.

¡Has realizado un excelente progreso en tu proyecto de **Transfer Learning**! Continúa fortaleciendo la **seguridad** y **funcionalidad** de tu API para ofrecer una experiencia robusta y segura a tus usuarios.

Si tienes alguna otra pregunta o necesitas más asistencia en los próximos pasos, **no dudes en contactarme**. ¡Estoy aquí para ayudarte a alcanzar el éxito en tu proyecto de **Transfer Learning**!